In [21]:
import re

In [22]:
def segmentation(line):
    chap, rest = line.split(':', maxsplit=1)
    meta = re.search('\d+? \[\w+?\] ', rest).group()
    m = len(meta)
    verse, stem = meta.split(' ', maxsplit=1)
    text = rest[m:]
    return [chap, verse, stem, text] 

In [23]:
def read_codex(fname, stop):
    with open(fname, 'r', encoding='utf-8') as f:
        full_text = f.read()
        full_text = re.sub('ю', "þ", full_text)
        full_text = re.sub('hv', 'ƕ', full_text)
        full_text = re.sub('ы', 'u', full_text) # û
        full_text = re.sub("п", "i", full_text) # ii
    lines = full_text.split('\n')
    chapters = {}
    books = set()
    while len(books) <= int(stop):
        for index, line in enumerate(lines):
            if not line.startswith('#') and line != '':
                parts = segmentation(line)
                chap = parts[0]
                books.add(chap.split(' ')[0])
                stem = parts[2]
                text_raw = parts[3]
                text = text_raw.lower()
                text = re.sub('[.,!?:;]', '', text)
                text = re.sub('(\s)\s+?', r'\1', text)
                if chap not in chapters:
                    verses_list = set([parts[2]])
                    i = index + 1
                    if i < len(lines):
                        n_line = lines[i]
                        if n_line != '':
                            nl_parts = segmentation(n_line)
                            verses_list.add(nl_parts[2])
                    while nl_parts[0] == chap:
                        i += 1
                        if i < len(lines):
                            n_line = lines[i]
                            if n_line != '':
                                nl_parts = segmentation(n_line)
                                if nl_parts[0] == chap:
                                    verses_list.add(nl_parts[2])
                        else:
                            break
                    if len(verses_list) == 1:
                        chapters[chap] = [text]
                    else:
                        chapters[chap] = dict()
                        for st in list(verses_list):
                            chapters[chap][st] = []
                        chapters[chap][stem].append(text)
                else:
                    if isinstance(chapters[chap],dict):
                        chapters[chap][stem].append(text)
                    else:
                        chapters[chap].append(text)
                
    return chapters

In [24]:
#stop = input('Сколько книг пройти? ')
stop = 10
#stop = 4
codex = read_codex('gotica.txt', stop)

In [25]:
def anafwords():
    with open('gotica.txt', 'r', encoding='utf-8') as f:
        codex = f.readlines()
    nwords = set()
    fwords = set()
    for line in codex:
        if not line.startswith('#'):
            found = re.findall('[Jj]ah ([Aa][nf][a-zþƕ]+)', line)
            if len(found) > 0:
                for word in found:
                    if word[1] == 'f':
                        fwords.add(word)
                    else:
                        nwords.add(word)
    with open('an-words.txt', 'w', encoding='utf-8') as fn:
        mess = '\n'.join(sorted(list(nwords)))
        fn.write(mess)
    with open('af-words.txt', 'w', encoding='utf-8') as ff:
        mess = '\n'.join(sorted(list(fwords)))
        ff.write(mess)

In [26]:
anafwords()

In [27]:
def letters_only(codex, command):
    letters = {}
    for key in codex:
        if isinstance(codex[key],dict) == False:
            buch = ''
            for verse in codex[key]:
                if ')' in command:
                    verse = re.sub('\([^)]+\)', '', verse)
                elif ']' in command:
                    verse = re.sub('\[[a-zþƕ]+\]', '', verse) # убрать то, что советовал убрать Шт.
                elif '>' in command:
                    verse = re.sub('<[a-zþƕ]+>', '', verse) # убрать то, что добавил Шт.
                raw = re.sub('[^a-zþƕ]', '', verse)
                buch += raw
            letters[key] = buch
        else:
            for stem in codex[key]:
                buch = ''
                for verse in codex[key][stem]:
                    raw = re.sub('[^a-zþƕ]', '', verse)
                    buch += raw
                n_key = key + ' ' + stem
                letters[n_key] = buch
    return letters

In [28]:
def search_for_fragments(letters, list_of_readings, p1, p2):
    joint = '.{' + str(p1)
    joint += ',' + str(p2)
    joint += '}'
    pattern = joint.join(list_of_readings)
    print(pattern)
    patt = re.compile(pattern, re.DOTALL)
    contain = []
    for chap in letters:
        text = letters[chap]
        if patt.search(text):
            print('ord')
            print(chap)
            print(list_of_readings)
            contain.append(chap)
    return chap

In [29]:
def search_two_lines(letters, list_of_readings, p1, p2):
    sub_patterns = {}
    for ind1, first in enumerate(list_of_readings):
        for ind2, second in enumerate(list_of_readings):
            if ind1 < ind2:
                diff = ind2 - ind1
                hole = diff - 1
                k1 = p1 * diff + 5 * hole
                k2 = p2 * diff + 5 * hole
                joint = '.{' + str(k1)
                joint += ',' + str(k2)
                joint += '}'
                pattern = first + joint + second
                print(pattern)
                patt = re.compile(pattern, re.DOTALL)
                for chap in letters:
                    text = letters[chap]
                    if patt.search(text):
                        print(chap)
                        sub_p = first + ' // ' + second
                        if sub_p in sub_patterns:
                            sub_patterns[sub_p].append(chap)
                        else:
                            sub_patterns[sub_p] = [chap]
    return sub_patterns

In [30]:
def fragments_free_order(letters, lor):
    l = len(lor)
    contain = []
    for chap in letters:
        i = 0
        text = letters[chap]
        for frag in lor:
            if re.search(frag, text):
                i += 1
        if i == l:
            print('free')
            print(chap)
            print(lor)
            contain.append(chap)
    return chap

In [31]:
#letters = letters_only(codex, 1)
readings = [
    ['swaswe', 'anssaþ', 'a?nþar'],
    ['aiþiu?', 'jahan', 'jahaf'],
    ['sa[rb]?þ', 'ain[ad]?', 'din'],
    ['ƕ?.þi', 'ehan', 'jaha'],
    ['þw', 'n.nþ?', 'il.n'],
]
p1 = 15
p2 = 50
#for liste in readings:
    #con = search_for_fragments(letters, liste, p1, p2)
    #con_f = fragments_free_order(letters, liste)

SyntaxError: unexpected EOF while parsing (<ipython-input-31-224ee1457126>, line 13)

In [32]:
letters_true = letters_only(codex, '>')
letters_stei = letters_only(codex, ']')
letters_stsu = letters_only(codex, '])')
letters = [letters_true, letters_stei, letters_stsu]
readings = [
    ['swaswe', 'n[sj]saei', 'anþ[al]r'],
    ['swaswe', 'n[sj]saþ', 'anþ[al]r'],
    ['aiþi[qu]', 'jahan', 'jahaf'],
    ['sar?þ', 'aina?', 'din'],
    ['ƕ?.þi', 'ehan', 'jaha'],
    ['þw', 'n.nþ', 'il.n']
]
p1 = 12
p2 = 55
for lett in letters:
    print('--')
    for liste in readings:
        con = search_for_fragments(lett, liste, p1, p2)

--
swaswe.{12,55}n[sj]saei.{12,55}anþ[al]r
swaswe.{12,55}n[sj]saþ.{12,55}anþ[al]r
aiþi[qu].{12,55}jahan.{12,55}jahaf
ord
Mt 6
['aiþi[qu]', 'jahan', 'jahaf']
sar?þ.{12,55}aina?.{12,55}din
ord
Mt 6
['sar?þ', 'aina?', 'din']
ord
Gal 2 [A] 
['sar?þ', 'aina?', 'din']
ƕ?.þi.{12,55}ehan.{12,55}jaha
þw.{12,55}n.nþ.{12,55}il.n
--
swaswe.{12,55}n[sj]saei.{12,55}anþ[al]r
swaswe.{12,55}n[sj]saþ.{12,55}anþ[al]r
aiþi[qu].{12,55}jahan.{12,55}jahaf
ord
Mt 6
['aiþi[qu]', 'jahan', 'jahaf']
sar?þ.{12,55}aina?.{12,55}din
ord
Mt 6
['sar?þ', 'aina?', 'din']
ord
Gal 2 [A] 
['sar?þ', 'aina?', 'din']
ƕ?.þi.{12,55}ehan.{12,55}jaha
þw.{12,55}n.nþ.{12,55}il.n
--
swaswe.{12,55}n[sj]saei.{12,55}anþ[al]r
swaswe.{12,55}n[sj]saþ.{12,55}anþ[al]r
aiþi[qu].{12,55}jahan.{12,55}jahaf
ord
Mt 6
['aiþi[qu]', 'jahan', 'jahaf']
sar?þ.{12,55}aina?.{12,55}din
ord
Mt 6
['sar?þ', 'aina?', 'din']
ord
Gal 2 [A] 
['sar?þ', 'aina?', 'din']
ƕ?.þi.{12,55}ehan.{12,55}jaha
þw.{12,55}n.nþ.{12,55}il.n


In [21]:
uncial = [
    ['a?usi?', 'atta', 'minþe', 'mi?seis'],
    ['eih.f', 'namm?', 'ij'],
    ['taiin', 'þanz?e?', 'nswa'],
    ['sinþom', 'wei?ha', 'naj']
]
p1 = 10
p2 = 25
for i in range(3):
    j = i*10
    p1 += j
    p2 += j
    for liste in uncial:
        con = search_for_fragments(letters_stsu, liste, p1, p2)
    #con_f = fragments_free_order(letters, liste)

a?usi?.{10,25}atta.{10,25}minþe.{10,25}mi?seis
ord
Jn 17
['a?usi?', 'atta', 'minþe', 'mi?seis']
eih.f.{10,25}namm?.{10,25}ij
ord
Jn 17
['eih.f', 'namm?', 'ij']
taiin.{10,25}þanz?e?.{10,25}nswa
ord
Jn 17
['taiin', 'þanz?e?', 'nswa']
sinþom.{10,25}wei?ha.{10,25}naj
ord
Jn 17
['sinþom', 'wei?ha', 'naj']
a?usi?.{20,35}atta.{20,35}minþe.{20,35}mi?seis
eih.f.{20,35}namm?.{20,35}ij
taiin.{20,35}þanz?e?.{20,35}nswa
sinþom.{20,35}wei?ha.{20,35}naj
a?usi?.{40,55}atta.{40,55}minþe.{40,55}mi?seis
eih.f.{40,55}namm?.{40,55}ij
taiin.{40,55}þanz?e?.{40,55}nswa
sinþom.{40,55}wei?ha.{40,55}naj


In [77]:
def get_the_verse(chap, phrase):
    with open('gotica.txt', 'r', encoding='utf-8') as f:
        holy_bible = f.read()
    if re.search('\d$', chap):
        pattern = chap + ':(\d\d?) \[\S+\] (.*?)\n'
    else:
        chaps = chap[:-1].rsplit(' ', maxsplit=1)
        pattern = chaps[0] + ':(\d\d?) \[' + chaps[1][1:-1] + '\] (.*?)\n'
    my_chap = re.findall(pattern, holy_bible)
    verses = []
    for index, ver in enumerate(my_chap):
        text = ver[1].lower()
        text = re.sub('<[a-zþƕ]+>', '', text) # убрать то, что добавил Шт.
        text = re.sub('[^a-zþƕ]', '', text)
        if re.search(phrase, text):
            verses.append(ver[0])
        try:
            next_ver = my_chap[index + 1][1].lower()
            next_ver = re.sub('<[a-zþƕ]+>', '', next_ver)
            next_ver = re.sub('[^a-zþƕ]', '', next_ver)
            for n in range(1, len(phrase)):
                if phrase[:n] == text[len(text)-n:]:
                    if next_ver.startswith(phrase[n:]):
                        n_ver = my_chap[index + 1][0]
                        number = str(ver[0]) + '–' + str(n_ver)
                        verses.append(number)
        except IndexError:
            break
    numbers = ','.join(verses)
    print(chap + ':' + numbers)
    return numbers

In [82]:
verse_check = ['swaswe', 'anþar', 'aiþiu', 'jahan', 'jahaf']
chapss = ['Mk 3', 'Mk 12', 'Lk 2', 'Lk 6', 'Lk 20', 'Jn 10']
def maketable3(chapss, verse_check):
    with open('table.tsv', 'w', encoding='utf-8') as f:
        f.write('\t' + '\t'.join(verse_check))
        f.write('\n')
        for ch in chapss:
            row = [ch]
            for frag in verse_check:
                print(frag)
                numbers = get_the_verse(ch, frag)
                if numbers:
                    row.append(numbers)
                else:
                    row.append('-')
            f.write('\t'.join(row))
            f.write('\n')

In [84]:
for ch in chapss:
    nuaa = get_the_verse(ch, 'jahanþar')

Mk 3:21
Mk 12:21,31
Lk 2:
Lk 6:17,29
Lk 20:
Jn 10:16


In [83]:
maketable3(chapss, verse_check)

swaswe
Mk 3:10,20,28
anþar
Mk 3:21
aiþiu
Mk 3:24
jahan
Mk 3:18,21,33
jahaf
Mk 3:
swaswe
Mk 12:
anþar
Mk 12:4,5,9,21,31,32
aiþiu
Mk 12:
jahan
Mk 12:1,17,21,24,31,35
jahaf
Mk 12:1,4,5,12
swaswe
Lk 2:20,23,24
anþar
Lk 2:
aiþiu
Lk 2:32
jahan
Lk 2:13,14,40,47,52
jahaf
Lk 2:
swaswe
Lk 6:10,31,36
anþar
Lk 6:1,6,10,17,29
aiþiu
Lk 6:33
jahan
Lk 6:3,14,17,29
jahaf
Lk 6:22,30
swaswe
Lk 20:
anþar
Lk 20:11,16,30
aiþiu
Lk 20:
jahan
Lk 20:7,9,11,12,34
jahaf
Lk 20:9,20
swaswe
Jn 10:15,26
anþar
Jn 10:16
aiþiu
Jn 10:
jahan
Jn 10:16
jahaf
Jn 10:


In [15]:
def if_in_chap(letters, single_lets, proportion):
    anth_wb = {}
    for chap in letters:
        text = letters[chap]
        for frag in single_lets:
            if re.search(frag, text):
                if chap not in anth_wb:
                    anth_wb[chap] = [frag]
                else:
                    anth_wb[chap].append(frag)
    itogo = len(single_lets)
    for ch in sorted(anth_wb, key=lambda ch: len(anth_wb[ch]), reverse=True):
        matched = len(anth_wb[ch])
        if matched / itogo >= proportion:
            message = ch + ' ' + str(matched) + ' ' + ';'.join(anth_wb[ch])
            print(message)
    return anth_wb

In [57]:
def found_rating(letters, single_lets):
    anth_wb = {}
    for chap in letters:
        text = letters[chap]
        for frag in single_lets:
            if re.search(frag, text):
                if frag not in anth_wb:
                    anth_wb[frag] = [chap]
                else:
                    anth_wb[frag].append(chap)
    itogo = len(letters)
    for ch in sorted(anth_wb, key=lambda ch: len(anth_wb[ch]), reverse=False):
        matched = len(anth_wb[ch])
        if matched < 15:
            message = ch + ' ' + str(matched) + ' ' + ';'.join(anth_wb[ch])
        else:
            message = ch + ' ' + str(matched)
        print(message)
    print(itogo)
    return anth_wb

In [65]:
single = [
    'swaswe', 
    'anssaþ', 'anþar',
    'aiþiu', 'jahan', 'jahaf',
    'sar?þ', 'aina?', 'din',
    'þw', 'n.nþ?', 'il.n'
]
another_wb = if_in_chap(letters_true, single, 0.5)

Mt 6 11 swaswe;anþar;aiþiu;jahan;jahaf;sar?þ;aina?;din;þw;n.nþ?;il.n
Lk 6 10 swaswe;anþar;aiþiu;jahan;jahaf;sar?þ;aina?;din;þw;n.nþ?
Mk 4 10 swaswe;anþar;jahan;jahaf;sar?þ;aina?;din;þw;n.nþ?;il.n
Mt 5 9 swaswe;anþar;aiþiu;jahan;sar?þ;aina?;din;n.nþ?;il.n
Mt 8 9 swaswe;anþar;jahan;jahaf;sar?þ;aina?;din;þw;n.nþ?
Jn 14 9 swaswe;anþar;jahan;sar?þ;aina?;din;þw;n.nþ?;il.n
Mk 10 9 swaswe;anþar;jahan;jahaf;aina?;din;þw;n.nþ?;il.n
1Cor 15 [A]  9 swaswe;anþar;jahan;jahaf;sar?þ;aina?;din;þw;n.nþ?
2Cor 1 [B]  9 swaswe;anþar;jahaf;sar?þ;aina?;din;þw;n.nþ?;il.n
Eph 3 [B]  9 swaswe;anþar;jahan;sar?þ;aina?;din;þw;n.nþ?;il.n
Mt 27 [CA]  8 swaswe;anþar;jahan;aina?;din;þw;n.nþ?;il.n
Jn 7 8 swaswe;anþar;jahaf;sar?þ;aina?;din;n.nþ?;il.n
Jn 13 8 swaswe;jahaf;sar?þ;aina?;din;þw;n.nþ?;il.n
Lk 3 8 swaswe;anþar;jahan;aina?;din;þw;n.nþ?;il.n
Lk 5 8 anþar;jahan;jahaf;aina?;din;þw;n.nþ?;il.n
Lk 7 8 anþar;jahan;sar?þ;aina?;din;þw;n.nþ?;il.n
Lk 18 8 swaswe;anþar;jahan;jahaf;aina?;din;þw;n.nþ?
Mk 1 8 swaswe;jahan;jah

In [37]:
def check_the_chapter(another_wb, letters):
    check = input('chapter number: ')
    while check not in another_wb:
        check = input('chapter number: ')
    print(len(another_wb[check]))
    print(another_wb[check])
    option = input('f = create file; p = print text; else buy ')
    if option.lower() == 'f' or option.lower() == 'p':
        text = letters[check]
        i = int(input('linelength: '))
        j = i
        lines = []
        while j < len(text):
            lines.append(text[j-i:j])
            j += i
        text[j-i:]
        if option.lower() == 'f':
            fname = input('fname: ')
            if not fname.endswith('.txt'):
                fname = check.strip() + '.txt'
            with open(fname, 'w', encoding='utf-8') as f:
                f.write('\n'.join(lines))
        elif option.lower() == 'p':
            for line in lines:
                print(line)

In [40]:
check_the_chapter(another_wb, letters_true)

chapter number: Mk 6
6
['a?nssaþ', 'a?nþar', 'sar?þ', 'aina?', 'din', 'n.nþ?']
f = create file; p = print text; else buy f
linelength: 37
fname: mk6.txt


In [62]:
rating = found_rating(letters_true, single)
for chapter in rating['aiþiu']:
    get_the_verse(chapter, 'aiþiu')

anssaþ 1 Mk 6
aiþiu 13 Mt 5;Mt 6;Jn 12;Jn 19;Lk 1;Lk 2;Lk 6;Mk 3;Eph 1 [B] ;Eph 1 [A] ;Gal 4 [B] ;1Thess 2;Phm 1
jahaf 41
il.n 50
sar?þ 65
jahan 89
anþar 97
swaswe 104
din 125
þw 142
aina? 189
n.nþ? 193
205
Mt 5:46
Mt 6:7,10
Jn 12:15,20
Jn 19:12
Lk 1:48
Lk 2:32
Lk 6:33
Mk 3:24
Eph 1 [B] :3
Eph 1 [A] :3
Gal 4 [B] :30
1Thess 2:12
Phm 1:14


In [66]:
cursive_big = [
    ['swaswe', 'a?nssaþ', 'anþar'],
    ['aiþiu', 'jahan', 'jahaf'],
    ['swaswe', 'a?nssaei', 'anþar']
]
for fragment in cursive_big:
    sub_patterns = search_two_lines(letters_true, fragment, 17, 50)

swaswe.{17,50}a?nssaþ
swaswe.{39,105}anþar
Jn 10
Mk 3
2Cor 1 [B] 
2Cor 1 [A] 
2Cor 8 [B] 
2Cor 8 [A] 
2Cor 13 [B] 
2Cor 13 [A] 
Gal 1 [B] 
1Tim 1 [B] 
1Tim 1 [A] 
a?nssaþ.{17,50}anþar
aiþiu.{17,50}jahan
Mt 6
aiþiu.{39,105}jahaf
Mt 6
jahan.{17,50}jahaf
Mt 6
Lk 20
Mk 12
swaswe.{17,50}a?nssaei
swaswe.{39,105}anþar
Jn 10
Mk 3
2Cor 1 [B] 
2Cor 1 [A] 
2Cor 8 [B] 
2Cor 8 [A] 
2Cor 13 [B] 
2Cor 13 [A] 
Gal 1 [B] 
1Tim 1 [B] 
1Tim 1 [A] 
a?nssaei.{17,50}anþar
Lk 15


In [21]:
for ch in sorted(another_wb, key=lambda ch: len(another_wb[ch]), reverse=True):
    print(ch)
    text = letters_true[ch]
    i = 23
    while i < len(text):
        print(text[i-23:i])
        i += 23
    print(text[i-23:len(text)])
    

Mt 6
atsaiƕiþarmaionizwarani
taujaninandwairþjamanne
dusaiƕanimaiþþaulaunnih
abaiþframattinizwaramma
þammainhiminamþannutauj
aisarmaionnihaurnjaisfa
uraþusswasweþailiutanst
aujandingaqumþimjahinga
runsimeihauhjaindaufram
mannamamenqiþaizwisandn
emunmizdonseinaiþþuktau
jandanarmaionniwitihlei
dumeiþeinaƕataujiþtaihs
woþeinaeisijaisoarmahai
rtiþaþeinainfulhsnjajah
attaþeinssaeisaiƕiþinfu
lhsnjausgibiþþusinbairh
teinjahþanbidjaiþnisija
iþswasweþailiutansuntef
rijondingaqumþimjahwaih
stamplapjostandandansbi
djaneigaumjaindaumannam
amenqiþaizwisþateihaban
dmizdonseinaiþþuþanbidj
aisgagginheþjonþeinajah
galukandshaurdaiþeinaib
ideiduattinþeinammaþamm
ainfulhsnjajahattaþeins
saeisaiƕiþinfulhsnjausg
ibiþþusinbairhteinbidja
ndansuþþannifiluwaurdja
iþswasweþaiþiudoþugkeiþ
imaukeiinfiluwaurdeinse
inaiandhausjaindaunigal
eikoþnuþaimwaitaukattai
zwarþizeijusþaurbuþfaur
þizeijusbidjaiþinaswanu
bidjaiþjusattaunsarþuin
himinamweihnainamoþeinq
imaiþiudinassusþeinswai
rþaiwiljaþeinssweinhimi
najahanaair

iþauhqimiþinizaijahgaþl
aihandsimlagjandshandun
sanaþoþiuþidaimjahusgag
gandinimmainwigduatrinn
andsainsjahknussjandsba
þinaqiþandslaisariþiuþe
igaƕataujaueilibainaisa
iweinonsarbjawairþauiþi
sqaþduimmaƕamikqiþisþiu
þeigananiƕashunþiuþeigs
aljaainsguþþosanabusnin
skantnihorinosnimaurþrj
aisnihlifaisnisijaisgal
iugaweitwodsnianamahtja
issweraiattanþeinanajah
aiþeinþeinaþaruhandhafj
andsqaþduimmalaisariþoa
llagafastaidausjundaime
inaiiþiesusinsaiƕandsdu
immafrijodainajahqaþdui
mmaainisþuswanistgaggsw
afiluswehabaisfrabugeij
ahgifþarbamjahhabaishuz
dinhiminamjahhirilaistj
anmiknimandsgalganiþisg
ahnipnandsinþiswaurdisg
alaiþgaurswasaukhabands
faihumanagjahbisaiƕands
iesusqaþsiponjamseinaim
saiƕaiwaaglubaþaifaihog
ahabandansinþiudangardj
agudisgaleiþandiþþaisip
onjosafslauþnodeduninwa
urdeisþaruhiesusaftraan
dhafjandsqaþimbarnilona
ƕaiwaagluistþaimhugjand
amafarfaihauinþiudangar
djagudisgaleiþanazitizo
istulbandauþairhþairkon
eþlosgaleiþanþaugabigam
mainþiudangardjagudisga
leiþaniþeismaisu

stinþuthaurnaþuthaurnei
þaukjahdauþansusstandan
dunriurjaijahweisinmaid
jandaskuldaukistþatariu
rjogahamonunriureinjahþ
atadiwanogahamonundiwan
einþanuþþanþatadiwanoga
wasjadaundiwaneinþanuhw
airþiþwaurdþatagamelido
ufsagqiþswarþdauþusinsi
gisƕaristgazdsþeinsdauþ
uƕaristsigisþeinhaljaaþ
þangazdsdauþausfrawaurh
tsiþmahtsfrawaurhtaiswi
toþiþgudaawiliuþizegafu
nsissigisþairhfraujanun
saranaiesuxristauswaein
ubroþrjusmeinailiubanst
ulgjaiwairþiþungawagida
iufarfulljandansinwaurs
twafraujinssinteinowita
ndansþateiarbaiþsizwara
nistlausainfraujin
2Cor 1 [B] 
dukaurinþaiumanþaradust
odeiþpawlusapaustaulusi
esuisxristausþairhwilja
ngudisjahteimauþaiusbro
þaraikklesjongudisþizai
wisandeininkaurinþonmiþ
allaimþaimweihamþaimwis
andaminallaiakaijaianst
sizwisjahgawairþiframgu
daattinunsarammajahfrau
jiniesuxristauþiuþiþsgu
þjahattafraujinsunsaris
iesuisxristausattableiþ
einojahguþallaizogaþlai
htesaeigaþrafstidaunsan
aallaiaglonunsaraieimag
eimaweisgaþrafstjanþans
inallaimaglomþairhþogaþ
laihtþiza

gardjaihimineiþsaeitauj
iþjahlaisjaiswasahmikil
shaitadainþiudangardjai
himineqiþaaukizwisþatei
nibaimanagizowairþiþizw
araizosgaraihteinsþauþi
zebokarjejahfareisaieni
þauqimiþinþiudangardjai
himinehausideduþþateiqi
þanistþaimairizamnimaur
þrjaisiþsaeimaurþreiþsk
ulawairþiþstauaiaþþanik
qiþaizwisþateiƕazuhmoda
gsbroþrseinammaswaresku
lawairþiþstauaiiþsaeiqi
þiþbroþrseinammarakasku
lawairþiþgaqumþaiaþþans
aeiqiþiþdwalaskulawairþ
iþingaiainnanfuninsjaba
inubairaisaibrþeinduhun
slastadajahjainargamune
isþateibroþarþeinshabai
þƕabiþukafletjainarþogi
baþeinainandwairþjahuns
lastadisjahgaggfaurþisg
asibjonbroþrþeinammajah
biþeatgaggandsatbairþog
ibaþeinasijaiswailahugj
andsandastauinþeinammas
prautoundþateiisinwigam
iþimmaibaiƕanatgibaiþuk
saandastauastauinjahsas
tauaþukatgibaiandbahtaj
ahinkarkaragalagjazaame
nqiþaþusniusgaggisjainþ
rounteusgibisþanaminnis
tankintuhausideduþþatei
qiþanistnihorinosaþþani
kqiþaizwisþateiƕazuhsae
isaiƕiþqinondulustonizo
sjugahorinodaizaiinhair
tinseinammaiþjab

þunsaistbisunjaisaprauf
etessumaihqeþunsaistxri
stussumaihqeþunibaiþauu
sgaleilaiaxristusqimiþn
iugameleinsqaþþateiusfr
aiwadaweidisjahusbeþlai
haimweihsaþareiwasdawei
dxristusqimiþþanuhmissa
qissinþizaimanageinwarþ
biinasumaihþanizewilded
unfahaninaakeiniainshun
uslagidaanainahandunsga
liþunþanþaiandbahtosduþ
aimauhumistamgudjamjahf
areisaiumþaruhqeþunduim
jainaiduƕeniattauhuþina
andhofunþaiandbahtosniƕ
anhunaiwrodidamannaswas
wesamannaandhofunþanimþ
aifareisaieisibaijahjus
afairzidaisijuþsaijauai
nshunþizereikegalaubide
diimmaaiþþaufareisaieal
jasomanageiþaieinikunnu
nwitoþfraqiþanaisindqaþ
nikaudemusduimsaeiatidd
jaduimmainnahtsumswisan
dsizeiibaiwitoþunsarsto
jiþmannannibaifaurþisha
useiþframimmajahufkunna
iþƕataujaiandhofunjahqe
þunduimmaibaijahþuusgal
eilaiaisussokeijahsaiƕþ
ateipraufetususgaleilai
aniurreisiþ
Jn 11
wasuhþansumssiukslazaru
safbeþaniasushaimaimarj
insjahmarþinsswistrsizo
swasuhþanmarjasoeisalbo
dafraujanbalsanajahbisw
arbfotunsisskuftaseinam
maþizozeibroþarlazarus

...

sadedeainaizoswitoþraid
idaazgonkalbonsgabranni
daizosutanabibaurgeinai
safaruhþanþoinwatowairp
andanshrainjahhwssoponj
ahwullairaudaiufartrusn
jandansswaswegadobþansu
farmitonmunandansiþioha
nnesidreigosdaupeinmeri
dajahmissadedeafletþaim
ainfalþabagawandjandamg
ahaihaitiþfraujaataflet
afrawaurhtejahfragiftwe
ihisahminsjahfragibands
imþateisunjusþiudangard
joswairþainaswaeisijaid
aupeinsiohannesanamidum
aitwaddjeligandeiufarþe
ihandeiraihtiswitodishr
aineiniþminizeifilausai
waggeljonsdaupeinaiinuh
þisbairhtabaunslaiseiþq
iþandsaþþanikinwatinizw
isdaupjaiþsaafarmisgagg
andaswinþozamisistþizei
ikniimwairþseianahneiwa
ndsandbindauskaudaraips
kohisissahþanizwisdaupe
iþinahminweihammabigare
hsnainu
Mt 7
taujainaizwismansswajah
justaujaiþimþataaukistw
itoþjahpraufeteisinngag
gaiþþairhaggwudaurunteb
raiddaurjahrumswigssabr
iggandainfralustaijahma
nagaisindþaiinngaleiþan
dansþairhþataƕanaggwuþa
tadaurjahþraihanswigssa
briggandainlibainaijahf
awaisindþaibigitandansþ
anaatsaiƕiþsweþauhfaura
liu

bammausbauhtaisijuþniwa
irþaiþskalkosmannamƕarj
izuhinþammeiatlaþoþswas
broþrjusinþammagastanda
iatgudaaþþanbimaujosana
busnfraujinsnihabaiþrag
ingibaswegaarmaiþsframf
raujindutriggwswisanman
nuþatagoþwisaninþizosan
dwairþonsþaurftaisþatei
goþistmannswawisangabun
dansisqenainisokeilausj
angalausiþsisqenainisok
eiqenaþþanjabainimisqen
nifrawaurhtesjahjabaili
ugadamawinifrawaurhtaiþ
aglonleikisgastaldandþo
swaleikaiþikizwisfreidj
a
1Cor 12
sumammuhskeireinsrazdoþ
atuþþanallwaurkeiþainsj
ahsasamaahmadaileiþsund
roƕarjammehswaswewilisw
eleikraihtisainistiþliþ
unshabaiþmanagansþaiþþa
nliþjusallaiusleikaþamm
aainammamanagaiwisandan
sainistleikswajahxristu
sjahaukinainammaahminwe
isallaiduainammaleikada
upidaisiumjaþþejudaieis
jaþþeþiudosjaþþeskalkos
jaþþefrijaijahallaiaina
mmaahmindragkidaisijumj
aþþanleiknistainsliþusa
kmanagaijabaiqiþaifotus
þateiniimhandusniimþisl
eikisnihatþammaleikanis
tusþammaleikajabaiqiþai
ausoþateiniimaugoniimþi
sleikisniatþammaleikani
stusþammaleikajabaialll
eikaug

2Thess 1 [B] 
duþaissalauneikaiumanþa
rapawlusjahsilbanusjaht
eimauþaiusaikklesjonþai
ssalauneikaieingudaatti
nunsarammajahfraujinies
uxristauanstsizwisjahga
wairþiframgudaattinunsa
rammajahfraujiniesuxris
tauawiliudonskulumgudas
inteinoinizwarabroþrjus
swaswewairþistunteufarw
ahseiþgalaubeinsizwaraj
ahmanagniþfriaþwaainƕar
jizuhallaizeizwarainizw
ismissoswaeiweissilbans
izwisƕopaminaikklesjomg
udisinstiwitjisizwarisj
ahgalaubeinaisinallaimw
rakjomizwaraimjahaglomþ
ozeiusþulaiþtaikngaraih
taizosstauosgudisduwair
þansbrigganizwisþiudang
ardjosgudisinþizozeijah
winniþsweþauh
2Thess 1 [A] 
aipistaulepawlausduþais
salauneikaiumbanastodei
þpawlusjahsilbanusjahte
imauþaiusaikklesjonþais
salauneikaieingudaattin
unsarammajahfraujiniesu
xristauanstsizwisjahgaw
airþiframgudaattinunsar
ammajahfraujiniesuxrist
auawiliudonskulumgudasi
nteinoinizwarabroþrjuss
waswewairþistunteufarwa
hseiþgalaubeinsizwaraja
hmanagniþfriaþwaainƕarj
izuhallaizeizwarainizwi
smissoswaeiweissilbansi
nizwisƕopaminaikklesjo

pawlusushramiþswarþiniz
waraaiþþauinnaminpawlus
daupidaiweseiþawiliudog
udaeiainnohunizwaranida
upidanibakrispujahgaiue
iƕasniqiþaiþateiinmeina
mmanamindaupidedjauikda
upidaaukjaþþansstaifana
usgadaukansþataanþarniw
aiteiainnohundaupidedja
uniþþaninsandidamikxris
tusdaupjanakwailamerjan
niinsnutreinwaurdiseini
lausjaidaugalgaxristaus
unteþatawaurdgalginsþai
mfralusnandamdwaliþaist
iþþaimganisandammahtsgu
disistgameliþistaukfraq
istjasnutreinþizesnutra
nejahfrodeinþizefrodane
uskiusaƕarhandugsƕarbok
areisƕarsokareisþisaiwi
snihdwalagatawidaguþhan
dugeinþisfairƕausunteau
kinhandugeingudisniufku
nnaidasafairƕusþairhhan
dugeinguþgaleikaidaguda
þairhþodwaliþaþizoswail
amereinaisganasjanþansg
alaubjandansunteiudaiei
staiknebidjandiþkrekosh
andugeinsokjandiþweisme
rjamiesuushramidanaiuda
iumgamarzeiniþþiudomdwa
liþaiþþaimgalaþodamiuda
iejahþiudoxristugudisma
htjahgudishandugeinunte
sodwaliþagudishandugoze
imannam
1Cor 5
jugastauidasweandwairþs
þanaswaþatagataujandani
nnaminfraujinsunsarisie
s

aisijuþnistjudaiusnihkr
eksnistskalksnihfreisni
stgumakundnihqinakundun
teallaijusainsijuþinxri
stauiesuaþþanþandejusxr
istausþannuabrahamisfra
iwsijuþjabbigahaitamarb
jans
Gal 5 [A] 
iþahmawiþraleikþonusism
issoandstandandeiniþisƕ
ahþateiwileiþþatataujiþ
aþþanjabaiahmintiuhanda
nisijuþufwitodaaþþanswi
kunþasindwaurstwaleikis
þateiisthorinassuskalki
nassusunhrainiþaaglaite
igaliugagudeskalkinassu
slubjaleiseifiaþwoshaif
steisaljanhatizajiukost
wistasseisbirodeinoshai
raiseisneiþamaurþradrug
kaneinsgabaurosjahþatag
aleikoþaimþateifauraqiþ
aizwisswejufauraqaþþate
iþaiþataswaleiktaujanda
nsþiudangardjosgudisarb
jansniwairþandiþakranah
minsistfrijaþwafaheþsga
wairþiusbeisneiseleible
iþeigalaubeinsqairreiga
hobainsswikneiwiþraþosw
aleikanistwitoþiþþaieis
indxristausleikseinushr
amidedunmiþwinnomjahlus
tumjabailibamahminahmin
jahgaggamniwairþaimafla
utaiunsmissoushaitandan
smissoinneiþawisandans
Php 2 [A] 
inþizehausideduþinasiuk
anjahauksiukswasneƕadau
þauakeiguþinagaarmaidaa
þþanniþatainei

In [ ]:
with open('alexa.html', 'r', encoding='utf-8') as f:
    code = f.readlines()
    new = code[2516:15229]
    full = code[15229:29413]
with open('alexa.txt', 'w', encoding='utf-8') as fr1:
    for line in new:
        if line != '' and line != '\n':
            fr1.write(line)
with open('alexa.txt', 'r', encoding='utf-8') as fr:
    evan = fr.read()
with open('alexa_full.txt', 'w', encoding='utf-8') as fr2:
    fr2.write(evan)
    for line in new:
        if line != '' and line != '\n':
            fr2.write(line)

In [16]:
vow_a = ['\u03b1', '\u03ac']
vow_i = ['\u0390', '\u03ca', '\u03af', '\u03b9']
vow_w = ['\u03c9', '\u03ce']
vow_o = ['\u03bf', '\u03cc']
vow_n = ['\u03ae', '\u03b7']
vow_u = ['\u03c5', '\u03b0', '\u03cb', '\u03cd']
vow_e = ['\u03ad', '\u03b5']
vowels = [vow_a, vow_i, vow_w, vow_o, vow_n, vow_u, vow_e]

In [18]:
def greek_clue(vowels, line):
    got, fle, gre = line.strip('\n').split('\t')
    all_vow = [i for item in vowels for i in item]
    patterns = []
    for word in gre.split(';'):
        if fle == 'I' and '-' not in word:
            if word[-2] in all_vow and len(word) > 4:
                qword = word[:-1] + '?' + word[-1] + '?'
            else:
                qword = word + '?'
        else:
            qword = word
        for vow in vowels:
            patt = '['
            patt += ''.join(vow)
            patt += ']'
            qword = re.sub(patt, patt, qword, flags=re.UNICODE)
        patterns.append(qword)
    return patterns

In [14]:
def greek_parsing(chapters, vowels, par1, par2, pref_list=[]):
    with open('piupiu2.txt', 'r', encoding='utf-8') as piu:
        wortschatz = piu.readlines()
    jah = '\sκ[αά][\u0390\u03ca\u03af\u03b9]\s'
    liste_jah = []
    if pref_list:
        for pr in pref_list:
            jah1 = jah + ' ?' + pr
            liste_jah.append(jah1)
    else:
        liste_jah = [jah, jah]
    joint = '.{' + str(par1) + ',' + str(par2) + '}'
    found_here = {}
    for art in wortschatz:
        words = greek_clue(vowels, art)
        for word in words:
            w = word + '\S{0,3}\s'
            pa = [w]
            pa.extend(liste_jah)
            patt = joint.join(pa)
            patt += '\s?\S+\s'
            for chapter in chapters:
                bibel = chapters[chapter]
                bibel = re.sub('(\S)\s\s+(\S)', r'\1 \2', bibel)
                found = re.findall(patt, bibel)
                if found:
                    outp = (word, found)
                    if chapter not in found_here:
                        found_here[chapter] = [outp]
                    else:
                        found_here[chapter].append(outp)
    return found_here

In [ ]:
def write_it_down(found_dict):
    with open(input('fname: '), 'w', encoding='utf-8') as f:
        for key in found_dict:
            exs = found_dict[key]
            f.write(key)
            f.write('\n')
            f.write('\n!! '.join(exs))
            f.write('\n\n')

In [136]:
def xmlparser():
    bezae = {}
    books = {}
    with open('bezae.xml', 'r', encoding='utf-8') as f:
        xml = f.read()
    b = re.findall('n="([A-Za-z]+)" type="book"><div [^>]*?n="(B0\d)', xml)
    for pair in b:
        if pair[0].startswith('J'):
            pr = 'Jn'
        elif pair[0].startswith('L'):
            pr = 'Lk'
        elif 'tt' in pair[0]:
            pr = 'Mt'
        elif pair[0] == 'Mark':
            pr = 'Mk'
        else:
            pr = 'Acts'
        books[pair[1]] = pr
    verses = xml.split('<ab ')
    for block in verses:
        meta = re.match('n="(B0\d)K(\d{1,3})V(\d{1,3})"', block)
        if meta:
            block = re.sub('<seg .*?</seg>', '', block, re.DOTALL)
            bkv = meta.group(1,2,3)
            eva = books[bkv[0]]
            chp = int(bkv[1])
            vrs = int(bkv[2])
            words = re.findall('<w[^>]*>([^<\n]+)\n?</w>', block)
            text = ' '.join(words)
            if eva not in bezae:
                bezae[eva] = dict()
            if chp not in bezae[eva]:
                bezae[eva][chp] = dict()
            bezae[eva][chp][vrs] = text
    with open('bezaeplain.txt', 'w', encoding='utf-8') as f:
        for book in bezae:
            for chp in sorted(bezae[book]):
                for vrs in sorted(bezae[book][chp]):
                    text = bezae[book][chp][vrs]
                    meta = book + ' ' + str(chp) + ':' + str(vrs)
                    line = meta + ' ' + text + '\n'
                    f.write(line)

In [137]:
xmlparser()

In [12]:
def greek_chapters():
    with open('bezaeplain.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
    greek = {}
    for line in lines:
        if line:
            chap, text = line.split(':', maxsplit=1)
            text = re.sub('\d\d? ', '', text)
            
            if text == '':
                text += '//'
            if chap not in greek:
                greek[chap] = text.strip('\n')
            else:
                greek[chap] += ' ' + text
    return greek

In [33]:
grecha = greek_chapters()
foundgr = greek_parsing(grecha, vowels, 20, 55)
with open('6c55_greek_ingot.txt', 'w', encoding='utf-8') as fg:
    with open('6c55_greek_notgot.txt', 'w', encoding='utf-8') as fo:
        for key in foundgr:
            if key in letters_true.keys():
                fg.write(key + '\n')
                for pair in foundgr[key]:
                    message = '\n'.join(pair[1])
                    fg.write(message + '\n')
            else:
                fo.write(key + '\n')
                for pair in foundgr[key]:
                    message = '\n'.join(pair[1])
                    fo.write(message + '\n')

In [7]:
def lengthofall(fname):
    with open(fname, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    words = {}
    common_len = 0
    for line in lines:
        if not line.startswith('#'):
            if re.match('[A-Z]\S+ \d\d?:\d{1,3} ', line):
                meta = re.match('[A-Z]\S+ \d\d?:\d{1,3} ', line).group()
                chapter, verse = meta.split(':')
                if chapter not in words:
                    words[chapter] = dict()
                verse = re.sub('\D', '', verse)
                text = re.sub('[A-Z]\S+ \d\d?:\d{1,3} ', '', line)
                if text.startswith('['):
                    text = re.sub('^\[[^\]]+\] ', '', text)
                text = re.sub('(\S)\s\s+(\S)', r'\1 \2', text)
                raw = text.strip()
                ww = raw.split(' ')
                common_len += len(ww)
                words[chapter][verse] = ww
    print(fname, common_len)
    return common_len, words

In [8]:
bez_len, bez_words = lengthofall('bezaeplain.txt')
wul_len, wul_words = lengthofall('gotica.txt')

bezaeplain.txt 75535
gotica.txt 51022


In [10]:
chap_len = {}
'(+ гот - гр)'
v_all = 0
kfs = []
for key in bez_words:
    if key in wul_words:
        w_verses = set(wul_words[key].keys())
        b_verses = set(bez_words[key].keys())
        if w_verses == b_verses:
            w_len = 0
            b_len = 0
            for ver in wul_words[key]:
                w_len += len(wul_words[key][ver])
                b_len += len(bez_words[key][ver])
            chap_len[key] = (w_len, b_len)
            dif = w_len - b_len
            vamount = len(wul_words[key].keys())
            koef = round(dif / vamount, 4)
            print(key, dif, vamount, koef)
            v_all += vamount
            kfs.append((vamount, koef))
statt = 0.0
for pair in kfs:
    statt += pair[0] * pair[1]
print(statt / v_all, v_all)

Jn 6 -155 71 -2.1831
Jn 9 -1 41 -0.0244
Jn 10 -42 42 -1.0
Jn 14 -29 31 -0.9355
Jn 15 -54 27 -2.0
Jn 16 -26 33 -0.7879
Jn 17 -58 26 -2.2308
Jn 18 -33 40 -0.825
Lk 2 -43 52 -0.8269
Lk 4 -13 44 -0.2955
Lk 5 -49 39 -1.2564
Lk 6 -46 49 -0.9388
Lk 7 16 50 0.32
Lk 8 -27 56 -0.4821
Lk 9 40 62 0.6452
Lk 15 -31 32 -0.9688
Lk 18 -28 43 -0.6512
Lk 19 12 48 0.25
Mk 2 9 28 0.3214
Mk 3 -38 35 -1.0857
Mk 4 -47 41 -1.1463
Mk 5 -13 43 -0.3023
Mk 7 -65 37 -1.7568
Mk 8 -31 38 -0.8158
Mk 9 -8 50 -0.16
Mk 10 -1 52 -0.0192
Mk 11 -10 33 -0.303
-0.6745401574803152 1143


In [35]:
def afancheck(vowels, fname):
    found_wb = {}
    checked_wb = {}
    with open('prfxs_use.txt', 'r', encoding='utf-8') as f:
        pfxs = f.read().split('\n')
    p_set = set()
    for line in pfxs:
        if '\t' in line:
            rang, pref, freq = line.split('\t', maxsplit=2)
            pref = re.sub('\?', '', pref)
            p_set.add(pref)
    p_list = list(p_set)
    with open(fname, 'r', encoding='utf-8') as f:
        found = f.read().split('\n')
        for line in found:
            if re.search('\d\d?', line):
                chapter = line
                found_wb[chapter] = list()
            else:
                words = line.split(' ')
                for index, word in enumerate(words):
                    if word == "και":
                        try:
                            ic = 0
                            for prefix in p_list:
                                if words[index + 1].startswith(prefix):
                                    ic += 1
                            if ic > 0:
                                if chapter in checked_wb:
                                    checked_wb[chapter].append(line)
                                else:
                                    checked_wb[chapter] = [line]
                        except IndexError:
                            pass
    amount = 0
    for key in checked_wb:
        amount += len(checked_wb[key])
    print(amount)
    for key in checked_wb:
        print(key, '\n', '\n'.join(checked_wb[key]))
    return checked_wb

In [36]:
checked_ingot = afancheck(vowels, '6c55_greek_ingot.txt')
checked_notgot = afancheck(vowels, '6c55_greek_notgot.txt')

4
Mk 4 
 καλην σπαρεντες οιτινες ακουουσιν τον λογον και παραδεχονται και καρποφορουσιν εν και εν και εν και ελεγεν 
Mk 10 
 βασιλειαν του &om; ως παιδιον ου μη εις αυτην εισελευσεται και προσκαλεσαμενος αυτα ετιθει τας χειρας επ αυτα και ευλογει 
Lk 19 
 βασιλευειν επ αυτους αγαγατε ωδε και κατασφαξατε ενπροσθεν μου και τον 
Mt 5 
 καλως ποιειτε τοις μεισουσιν ϋμας και προσευχεσθαι ϋπερ των και διωκοντων 
3
Mt 21 
 ληστων και προσηλθον αυτω τυφλοι και χωλοι εν τω ϊερω και εθεραπευσεν αυτους ειδοντες δε οι αρχιερεις και οι 
Lk 23 
 βασιλειαν του &om; και προσελθων τω πειλατω ητησατο το σωμα του και καθελων ενετυλιξεν το σωμα του εν σινδονι και εθηκεν 
Mt 26 
 παισας σε ο δε πετρος εκαθητο εξω εν τη αυλη και προσηλθεν αυτω μια παιδισκη λεγουσα και συ 
